In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import pickle
from multiprocessing import Pool
from runpy import run_path

import time
import sys

ABPRED_DIR = Path.cwd().parent
if ABPRED_DIR not in sys.path:
    sys.path.append(str(ABPRED_DIR))
    
from AbPred.interface_contacts import InterfaceContacts
from AbPred.features_ic import ArpeggioFeatures,features15,features300,features400
from AbPred.FoldX import FoldX
from features3D.Structural import run_pops
from features3D.Structural import POPS_parse2
from AbPred.FoldX import names_stability_complex_wt,names_stability_complex_mut
from AbPred.FoldX import read_analyse_complex
from AbPred.FoldX import convert_features_to_differences
from biopandas.pdb import PandasPdb
import prody


/mnt/d/Python_projects/AbPred/AbPred/../libs/arpeggio


In [2]:
CWD = Path.cwd()


# PDBs dir
PDBS_WT = Path("../benchmark/vih_wt/")
PDBS_MUT = Path("../benchmark/vih_mut/")
# Dataframe , sort to match pdbfiles order
vih_dataframe = pd.read_csv('../data/HIV_escape_mutations.csv',sep="\t")


# PDB files
pdbs_mut = list(PDBS_MUT.glob('*Repair4.pdb'))
pdbs_wt = list(PDBS_WT.glob('*Repair4.pdb'))

In [3]:
vih_dataframe

,Mutation_type,Chain,Mutation,log10(Exptal),Exptal,mCSM-AB_Pred
0,ORIGINAL,G,A54E,1.995635,98.0000,-0.013
1,ORIGINAL,G,A62M,1.919078,82.0000,-0.761
2,ORIGINAL,G,A64K,1.949390,88.0000,-0.167
3,ORIGINAL,G,A69E,2.004321,100.0000,-0.610
4,ORIGINAL,G,A79W,2.025306,105.0000,-0.246
5,ORIGINAL,G,A87V,2.164353,145.0000,-0.063
6,ORIGINAL,G,A88K,2.056905,113.0000,-0.075
7,ORIGINAL,G,A89L,2.021189,104.0000,-0.208
8,ORIGINAL,G,A163N,1.863323,72.0000,-0.010
9,ORIGINAL,G,K163N,1.973128,93.0000,-0.299


# Clean structures to run arpeggio

In [4]:
TOOLS_DIR = ABPRED_DIR/"libs/tools"
CLEANPDB = TOOLS_DIR / "clean_pdb.py"

In [7]:
def run_cleanpdb(pdb):
    sys.argv = ["",pdb]
    run_path(str(CLEANPDB),run_name="__main__")

In [8]:
with Pool(12) as p:
    print(p.map(run_cleanpdb, [str(pdb) for pdb in pdbs_mut]))

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  war

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=O) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=O) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=O) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=O) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=O) with given element ''
  war

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CD) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CD) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CD) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OE1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OE1) with given element '

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CG1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CG1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CG2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CG2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CG2) with given elemen

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OG) with given element ''


...

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CH2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OD1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OD2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OG1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=ND2) with given elemen

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OH) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=C) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OD1) with given element ''
 

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=ND2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=NE1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CE3) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CZ2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element 

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CZ2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CZ3) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CH2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OXT) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=ND1) with given elemen

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [12]:
pdbs_clean_mut = list(PDBS_MUT.glob('*clean.pdb'))
pdbs_clean_mut

[]

In [10]:
# wildtype
with Pool(12) as p:
    print(p.map(run_cleanpdb, [str(pdb) for pdb in pdbs_wt]))

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  war

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=O) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CB) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CB) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CB) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CB) with given element ''
 

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OE1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OE1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=NE2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=C) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=NE2) with given element 

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CG2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CD1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CD1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CD1) with given element

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CG1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'S' for Atom (name=SD) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'S' for Atom (name=SD) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CG2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'S' for Atom (name=SD) with given element '

...

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OH) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OD1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OD2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OG1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=ND2) with given element

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OH) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OD1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OD2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OG1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=ND2) with given element

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CD2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OE2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=NE) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'S' for Atom (name=SD) with given element '

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CZ3) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CH2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=ND1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=ND1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OXT) with given elemen

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [8]:
pdbs_clean_wt = list(PDBS_WT.glob('*clean.pdb'))


In [9]:
len(pdbs_clean_wt)

78

# Calcule contact data mutant

In [15]:
pdbs_clean_wt[:1]

[PosixPath('../benchmark/vih_wt/VRC01.protein.G163A.mut.G163N.Repair4.clean.pdb')]

In [10]:
def main(i,pdbfile):
    
    partners_pdbs = 'HL_G'
    ic = InterfaceContacts(pdbfile)
    print("### Running arpeggio ###")
    print("Getting contact on ", pdbfile.name)
    ic.run_arpeggio()
    time.sleep(5)
    ic.get_ic(partners=partners_pdbs)
    print("Finish getting interface contact\n")
    ic.get_pymol_contacts()
    print("### END ###\n")

In [16]:
with Pool(2) as p:
    print(p.starmap(main, enumerate(pdbs_clean_mut)))

### Running arpeggio ###
### Running arpeggio ###
### Running arpeggio ###
### Running arpeggio ###
### Running arpeggio ###
### Running arpeggio ###
### Running arpeggio ###
### Running arpeggio ###
### Running arpeggio ###
### Running arpeggio ###
### Running arpeggio ###
Getting contact on  VRC01.protein.G168A.Repair4.clean.pdb
Getting contact on  VRC01.protein.G163A.Repair4.clean.pdb
Getting contact on  VRC01.protein.G163T.Repair4.clean.pdb
Getting contact on  VRC01.protein.G228A.Repair4.clean.pdb
### Running arpeggio ###
Getting contact on  VRC01.protein.G179A.Repair4.clean.pdb
Getting contact on  VRC01.protein.G222A.Repair4.clean.pdb
Getting contact on  VRC01.protein.G242A.Repair4.clean.pdb
Getting contact on  VRC01.protein.G246A.Repair4.clean.pdb
Getting contact on  VRC01.protein.G249A.Repair4.clean.pdb
Getting contact on  VRC01.protein.G264A.Repair4.clean.pdb
Getting contact on  VRC01.protein.G290A.Repair4.clean.pdb
Getting contact on  VRC01.protein.G295A.Repair4.clean.pdb
Gett

Getting contact on  VRC01.protein.G431A.Repair4.clean.pdb
Finish getting interface contact

### END ###

### Running arpeggio ###
Getting contact on  VRC01.protein.G433A.Repair4.clean.pdb
Finish getting interface contact
Finish getting interface contact


### END ###
### END ###


### Running arpeggio ###
### Running arpeggio ###
Getting contact on  VRC01.protein.G436A.Repair4.clean.pdb
Getting contact on  VRC01.protein.G439A.Repair4.clean.pdb
Finish getting interface contact

### END ###

### Running arpeggio ###
Getting contact on  VRC01.protein.G54A.Repair4.clean.pdb
Getting only interface contacts
Finish getting interface contact

### END ###

### Running arpeggio ###
Getting contact on  VRC01.protein.G392A.Repair4.clean.pdb
Getting only interface contacts
Finish getting interface contact

### END ###

### Running arpeggio ###
Getting contact on  VRC01.protein.G415A.Repair4.clean.pdb
Getting only interface contacts
Finish getting interface contact

### END ###

### Running arpeggio

In [17]:
# check pdb procced
ic_filesMUT = list(PDBS_MUT.glob("*.interface_contacts"))
len(ic_filesMUT)

78

In [16]:
# wildtype
with Pool(2) as p:
    print(p.starmap(main, enumerate(pdbs_clean_wt[:1])))

### Running arpeggio ###
Getting contact on  VRC01.protein.G163A.mut.G163N.Repair4.clean.pdb
Getting only interface contacts
Finish getting interface contact

Creating file VRC01.protein.G163A.mut.G163N.Repair4.clean.pml
Drew 644 total contacts.
### END ###

[None]


In [19]:
# check pdb procced
ic_filesWT = list(PDBS_WT.glob("*.interface_contacts"))
len(ic_filesWT)

78

# Extract contact features 
#### Mutant

In [20]:
def contact_features_(ic_files,pdbs_files,level_contacts='normal'):
    ''' Get interface contact features.
    
    Attributes
    ----------
    level_contact: str
        Level of interface description calculated from ic_file.
        This can be `normal`(15 features), `residue`(300 features) or `pairs`(400 features) '''
    
    if level_contacts == 'normal':
        features = np.empty((len(pdbs_files),15))
        ic_features = ArpeggioFeatures(level_sift="normal")
        
    elif level_contacts == 'residue':
        features = np.empty((len(pdbs_files),300))
        ic_features = ArpeggioFeatures(level_sift="residue")
    elif level_contacts == 'pairs':
        features = np.empty((len(pdbs_files),400))
        ic_features = ArpeggioFeatures(level_sift="pairs")
        
    for i,(ic,pdb) in enumerate(zip(ic_files,pdbs_files)):
        print(ic,pdb)
        ic_features.parse_contactdata(icfile=ic)
        ic_features.parse_pdbdata(pdbfile=pdb)

        contacts = ic_features.get_features()


        features[i] = contacts
        
    return features

In [21]:
data15 = contact_features_(ic_filesMUT,pdbs_clean_mut,level_contacts='normal')

../benchmark/vih_mut/VRC01.protein.G163A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G163A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G163K.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G163K.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G163T.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G163T.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G165A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G165A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G168A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G168A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G176A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G176A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G179A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G179A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G218A.Repair4.clean.interface_cont

../benchmark/vih_mut/VRC01.protein.G431A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G431A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G432A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G432A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G433A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G433A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G434A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G434A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G436A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G436A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G437A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G437A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G439A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G439A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G440A.Repair4.clean.interface_cont

In [22]:
DF_contact15_mut = pd.DataFrame(data15, index=[pdb.name for pdb in pdbs_clean_mut],columns=features15)

In [23]:
DF_contact15_mut.to_csv('../benchmark/VIH_contact15_mut.csv')

In [24]:
data300 = contact_features_(ic_filesMUT,pdbs_clean_mut,level_contacts='residue')

../benchmark/vih_mut/VRC01.protein.G163A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G163A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G163K.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G163K.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G163T.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G163T.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G165A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G165A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G168A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G168A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G176A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G176A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G179A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G179A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G218A.Repair4.clean.interface_cont

../benchmark/vih_mut/VRC01.protein.G431A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G431A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G432A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G432A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G433A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G433A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G434A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G434A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G436A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G436A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G437A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G437A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G439A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G439A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G440A.Repair4.clean.interface_cont

In [25]:
DF_contact300_mut = pd.DataFrame(data300, index=[pdb.name for pdb in pdbs_clean_mut],columns=features300)

In [26]:
DF_contact300_mut.to_csv('../benchmark/VIH_contact300_mut.csv')

In [27]:
data400 = contact_features_(ic_filesMUT,pdbs_clean_mut,level_contacts='pairs')

../benchmark/vih_mut/VRC01.protein.G163A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G163A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G163K.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G163K.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G163T.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G163T.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G165A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G165A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G168A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G168A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G176A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G176A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G179A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G179A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G218A.Repair4.clean.interface_cont

../benchmark/vih_mut/VRC01.protein.G431A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G431A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G432A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G432A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G433A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G433A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G434A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G434A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G436A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G436A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G437A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G437A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G439A.Repair4.clean.interface_contacts ../benchmark/vih_mut/VRC01.protein.G439A.Repair4.clean.pdb
../benchmark/vih_mut/VRC01.protein.G440A.Repair4.clean.interface_cont

In [28]:
DF_contact400_mut = pd.DataFrame(data400, index=[pdb.name for pdb in pdbs_clean_mut],columns=features400)

In [29]:
DF_contact400_mut.to_csv('../benchmark/VIH_contact400_mut.csv')

#### Wildtype

In [30]:
data15wt = contact_features_(ic_filesWT,pdbs_clean_wt,level_contacts='normal')

../benchmark/vih_wt/VRC01.protein.G163A.mut.G163N.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G163A.mut.G163N.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G163K.mut.G163N.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G163K.mut.G163N.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G163T.mut.G163N.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G163T.mut.G163N.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G165A.mut.G165S.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G165A.mut.G165S.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G168A.mut.G168T.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G168A.mut.G168T.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G176A.mut.G176F.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G176A.mut.G176F.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G179A.mut.G179I.Repair4.clean.interface_contacts ../benchmar

../benchmark/vih_wt/VRC01.protein.G417A.mut.G417D.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G417A.mut.G417D.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G418A.mut.G418G.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G418A.mut.G418G.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G419A.mut.G419G.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G419A.mut.G419G.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G421A.mut.G421N.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G421A.mut.G421N.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G423A.mut.G423S.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G423A.mut.G423S.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G427A.mut.G427I.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G427A.mut.G427I.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G429A.mut.G429R.Repair4.clean.interface_contacts ../benchmar

In [31]:
DF_contact15_wt = pd.DataFrame(data15wt, index=[pdb.name for pdb in pdbs_clean_wt],columns=features15)

In [32]:
DF_contact15_wt.to_csv('../benchmark/VIH_contact15_wt.csv')

In [33]:
data300wt = contact_features_(ic_filesWT,pdbs_clean_wt,level_contacts='residue')

../benchmark/vih_wt/VRC01.protein.G163A.mut.G163N.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G163A.mut.G163N.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G163K.mut.G163N.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G163K.mut.G163N.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G163T.mut.G163N.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G163T.mut.G163N.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G165A.mut.G165S.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G165A.mut.G165S.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G168A.mut.G168T.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G168A.mut.G168T.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G176A.mut.G176F.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G176A.mut.G176F.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G179A.mut.G179I.Repair4.clean.interface_contacts ../benchmar

../benchmark/vih_wt/VRC01.protein.G417A.mut.G417D.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G417A.mut.G417D.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G418A.mut.G418G.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G418A.mut.G418G.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G419A.mut.G419G.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G419A.mut.G419G.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G421A.mut.G421N.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G421A.mut.G421N.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G423A.mut.G423S.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G423A.mut.G423S.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G427A.mut.G427I.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G427A.mut.G427I.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G429A.mut.G429R.Repair4.clean.interface_contacts ../benchmar

In [34]:
DF_contact300_wt = pd.DataFrame(data300wt, index=[pdb.name for pdb in pdbs_clean_wt],columns=features300)

In [35]:
DF_contact300_wt.to_csv('../benchmark/VIH_contact300_wt.csv')

In [36]:
data400wt = contact_features_(ic_filesWT,pdbs_clean_wt,level_contacts='pairs')

../benchmark/vih_wt/VRC01.protein.G163A.mut.G163N.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G163A.mut.G163N.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G163K.mut.G163N.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G163K.mut.G163N.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G163T.mut.G163N.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G163T.mut.G163N.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G165A.mut.G165S.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G165A.mut.G165S.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G168A.mut.G168T.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G168A.mut.G168T.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G176A.mut.G176F.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G176A.mut.G176F.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G179A.mut.G179I.Repair4.clean.interface_contacts ../benchmar

../benchmark/vih_wt/VRC01.protein.G417A.mut.G417D.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G417A.mut.G417D.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G418A.mut.G418G.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G418A.mut.G418G.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G419A.mut.G419G.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G419A.mut.G419G.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G421A.mut.G421N.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G421A.mut.G421N.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G423A.mut.G423S.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G423A.mut.G423S.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G427A.mut.G427I.Repair4.clean.interface_contacts ../benchmark/vih_wt/VRC01.protein.G427A.mut.G427I.Repair4.clean.pdb
../benchmark/vih_wt/VRC01.protein.G429A.mut.G429R.Repair4.clean.interface_contacts ../benchmar

In [37]:
DF_contact400_wt = pd.DataFrame(data400wt, index=[pdb.name for pdb in pdbs_clean_wt],columns=features400)

In [38]:
DF_contact400_wt.to_csv('../benchmark/VIH_contact400_wt.csv')

# Calculate features differences .
## Contact
## mutant -> wildtype (wt - mut) and  wildtype -> mutant  (mut - wt)

In [132]:
DF_contact15_mut = pd.read_csv('../benchmark/VIH_contact15_mut.csv',index_col=0)
DF_contact15_wt = pd.read_csv('../benchmark/VIH_contact15_wt.csv',index_col=0)

DF_contact300_mut = pd.read_csv('../benchmark/VIH_contact300_mut.csv',index_col=0)
DF_contact300_wt = pd.read_csv('../benchmark/VIH_contact300_wt.csv',index_col=0)

DF_contact400_mut = pd.read_csv('../benchmark/VIH_contact400_mut.csv',index_col=0)
DF_contact400_wt = pd.read_csv('../benchmark/VIH_contact400_wt.csv',index_col=0)

In [135]:
# 15 contact
DF_contact15change_wt_mut =  DF_contact15_mut.values - DF_contact15_wt.values
col_changes = DF_contact15_mut.columns + '_change'
DF_contact15change_wt_mut = pd.DataFrame(DF_contact15change_wt_mut,index=[pdb.name for pdb in pdbs_clean_mut],columns=col_changes)

In [41]:
# 300 contact
DF_contact300change_wt_mut =  DF_contact300_mut.values - DF_contact300_wt.values
col_changes300 = DF_contact300_mut.columns + '_change'
DF_contact300change_wt_mut = pd.DataFrame(DF_contact300change_wt_mut,index=[pdb.name for pdb in pdbs_clean_mut],columns=col_changes300)

In [42]:
# 400 contact
DF_contact400change_wt_mut =  DF_contact400_mut.values - DF_contact400_wt.values
col_changes400 = DF_contact400_mut.columns + '_change'
DF_contact400change_wt_mut = pd.DataFrame(DF_contact400change_wt_mut,index=[pdb.name for pdb in pdbs_clean_mut],columns=col_changes400)

In [139]:
DF_contact15change_wt_mut.to_csv('../benchmark/VIH_contact15change_wt_mut.csv')
DF_contact300change_wt_mut.to_csv('../benchmark/VIH_contact300change_wt_mut.csv')
DF_contact400change_wt_mut.to_csv('../benchmark/VIH_contact400change_wt_mut.csv')

In [44]:
DF_contact15change_mut_wt = DF_contact15_wt.values - DF_contact15_mut.values
DF_contact15change_mut_wt = pd.DataFrame(DF_contact15change_mut_wt,index=[pdb.name for pdb in pdbs_clean_wt],columns=col_changes)

In [45]:
DF_contact300change_mut_wt = DF_contact300_wt.values - DF_contact300_mut.values
DF_contact300change_mut_wt = pd.DataFrame(DF_contact300change_mut_wt,index=[pdb.name for pdb in pdbs_clean_wt],columns=col_changes300)

In [46]:
DF_contact400change_mut_wt = DF_contact400_wt.values - DF_contact400_mut.values
DF_contact400change_mut_wt = pd.DataFrame(DF_contact400change_mut_wt,index=[pdb.name for pdb in pdbs_clean_wt],columns=col_changes400)

In [61]:
DF_contact15change_mut_wt.to_csv('../benchmark/VIH_contact15change_mut_wt.csv')
DF_contact300change_mut_wt.to_csv('../benchmark/VIH_contact300change_mut_wt.csv')
DF_contact400change_mut_wt.to_csv('../benchmark/VIH_contact400change_mut_wt.csv')

# Energetic features 
## MUTANT

In [49]:
def mainEnergetic(i,pdb):
    
    partners = 'HL_G'
    foldx = FoldX(verbose=False)
    result_analyse = foldx.analyse_complex(pdb_file=pdb.name, partners=partners)
    #energetic_featuresMUT[i] = result_analyse
    
    return result_analyse

In [50]:
try:
    os.chdir(PDBS_MUT)
    with Pool(12) as p:
        result_analyseMUT = p.starmap(mainEnergetic, enumerate(pdbs_clean_mut))
finally:
    os.chdir(CWD)


foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
f

In [51]:
energetic_featuresMUT = np.asarray(result_analyseMUT)

In [52]:
energetic_dataMUT = pd.DataFrame(energetic_featuresMUT, index=[pdb.name for pdb in pdbs_clean_mut],columns=names_stability_complex_mut)

In [62]:
energetic_dataMUT.to_csv("../benchmark/VIH_energetic_mut.csv")

# WT

In [57]:
pdbs_clean_wt

[PosixPath('../benchmark/vih_wt/VRC01.protein.G163A.mut.G163N.Repair4.clean.pdb'),
 PosixPath('../benchmark/vih_wt/VRC01.protein.G163K.mut.G163N.Repair4.clean.pdb'),
 PosixPath('../benchmark/vih_wt/VRC01.protein.G163T.mut.G163N.Repair4.clean.pdb'),
 PosixPath('../benchmark/vih_wt/VRC01.protein.G165A.mut.G165S.Repair4.clean.pdb'),
 PosixPath('../benchmark/vih_wt/VRC01.protein.G168A.mut.G168T.Repair4.clean.pdb'),
 PosixPath('../benchmark/vih_wt/VRC01.protein.G176A.mut.G176F.Repair4.clean.pdb'),
 PosixPath('../benchmark/vih_wt/VRC01.protein.G179A.mut.G179I.Repair4.clean.pdb'),
 PosixPath('../benchmark/vih_wt/VRC01.protein.G218A.mut.G218R.Repair4.clean.pdb'),
 PosixPath('../benchmark/vih_wt/VRC01.protein.G222A.mut.G222S.Repair4.clean.pdb'),
 PosixPath('../benchmark/vih_wt/VRC01.protein.G223A.mut.G223T.Repair4.clean.pdb'),
 PosixPath('../benchmark/vih_wt/VRC01.protein.G228A.mut.G228N.Repair4.clean.pdb'),
 PosixPath('../benchmark/vih_wt/VRC01.protein.G239A.mut.G239R.Repair4.clean.pdb'),
 Pos

In [58]:
try:
    os.chdir(PDBS_WT)
    with Pool(12) as p:
        result_analyseWT = p.starmap(mainEnergetic, enumerate(pdbs_clean_wt))
finally:
    os.chdir(CWD)

foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx path exported
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
foldx bin folder already in PATH
f

In [59]:
energetic_featuresWT = np.asarray(result_analyseWT)

In [60]:
energetic_dataWT = pd.DataFrame(energetic_featuresWT, index=[pdb.name for pdb in pdbs_clean_wt],columns=names_stability_complex_wt)

In [63]:
energetic_dataWT.to_csv("../benchmark/VIH_energetic_wt.csv")

## Energetic differences
## mutant -> wildtype (wt - mut) and  wildtype -> mutant  (mut - wt)

In [9]:
DF_energetic_wt = pd.read_csv("../benchmark/VIH_energetic_wt.csv",index_col=0)
DF_energetic_mut = pd.read_csv("../benchmark/VIH_energetic_mut.csv",index_col=0)

In [ ]:
DF_energetic_mut

In [10]:
# wildtype -> mutant (mut - wt)
DF_energetic_change_wt_mut =  DF_energetic_mut.values - DF_energetic_wt.values
col_changes1 = DF_energetic_mut.columns.str.replace('_mut','_change')
# DF energetic change and parent(wt) 
DF_energetic_change_wt_mut = pd.DataFrame(np.concatenate([DF_energetic_change_wt_mut,DF_energetic_mut.values],axis=1),\
index=[pdb.name for pdb in pdbs_clean_mut],columns=col_changes1.append(DF_energetic_mut.columns))

In [12]:
DF_energetic_change_wt_mut.to_csv("../benchmark/VIH_energetic_change_wt_mut.csv")

In [13]:
# mutant -> wildtype (wt - mut)
DF_energetic_change_mut_wt =  DF_energetic_wt.values - DF_energetic_mut.values
col_changes2 = DF_energetic_wt.columns.str.replace('_wt','_change')
# DF energetic change and parent(mut) 
DF_energetic_change_mut_wt = pd.DataFrame(np.concatenate([DF_energetic_change_mut_wt,DF_energetic_wt.values],axis=1),\
index=[pdb.name for pdb in pdbs_clean_wt],columns=col_changes2.append(DF_energetic_mut.columns))

In [16]:
DF_energetic_change_mut_wt.to_csv("../benchmark/VIH_energetic_change_mut_wt.csv")

# SASA features

In [71]:
def get_pdb_chain(pdb,partner,outdir):
    protein = prody.parsePDB(pdb,chain=partner)
    pdbwrited = prody.writePDB(str(outdir.resolve())+"/"+"%s_%s" % (pdb.name[:-4], partner), protein)
    print("writing %s_%s" % (pdb.name[:-4], partner))
    return pdbwrited

In [74]:
# MUtant interface sasas calculate as (TA + TB - T) / 2.0.
# When T=total sas complex, TA= total sasa partner A , TB=total sasa partner B.

outfolder = Path('popout_VIH')
if not outfolder.exists():
    outfolder.mkdir()
complexMUT_sasas = list()
partnerA_MUT_sasas = list()
partnerB_MUT_sasas = list()
for i,pdb in enumerate(pdbs_clean_mut):
    
    pp = 'HL_G'
    
    # Calculate sasa for complex
    asa_data = run_pops(str(pdb.resolve()), outdir=outfolder, mode='total')
    pops_complex = POPS_parse2(asa_data, mode='total') 
    complexMUT_sasas.append(list(pops_complex.values()))
    
    # Calculate sasa for partner A
    pdbA = get_pdb_chain(pdb.resolve(),partner=pp.split("_")[0],outdir=outfolder)
    asa_data = run_pops(pdbA, outdir=outfolder, mode='total')
    pops_A = POPS_parse2(asa_data, mode='total') 
    partnerA_MUT_sasas.append(list(pops_A.values()))
    
    # Calculate sasa for partner B
    pdbB = get_pdb_chain(pdb.resolve(),partner=pp.split("_")[1],outdir=outfolder)
    asa_data = run_pops(pdbB, outdir=outfolder, mode='total')
    pops_B = POPS_parse2(asa_data, mode='total') 
    partnerB_MUT_sasas.append(list(pops_B.values()))

writing VRC01.protein.G163A.Repair4.clean_HL
writing VRC01.protein.G163A.Repair4.clean_G
writing VRC01.protein.G163K.Repair4.clean_HL
writing VRC01.protein.G163K.Repair4.clean_G
writing VRC01.protein.G163T.Repair4.clean_HL
writing VRC01.protein.G163T.Repair4.clean_G
writing VRC01.protein.G165A.Repair4.clean_HL
writing VRC01.protein.G165A.Repair4.clean_G
writing VRC01.protein.G168A.Repair4.clean_HL
writing VRC01.protein.G168A.Repair4.clean_G
writing VRC01.protein.G176A.Repair4.clean_HL
writing VRC01.protein.G176A.Repair4.clean_G
writing VRC01.protein.G179A.Repair4.clean_HL
writing VRC01.protein.G179A.Repair4.clean_G
writing VRC01.protein.G218A.Repair4.clean_HL
writing VRC01.protein.G218A.Repair4.clean_G
writing VRC01.protein.G222A.Repair4.clean_HL
writing VRC01.protein.G222A.Repair4.clean_G
writing VRC01.protein.G223A.Repair4.clean_HL
writing VRC01.protein.G223A.Repair4.clean_G
writing VRC01.protein.G228A.Repair4.clean_HL
writing VRC01.protein.G228A.Repair4.clean_G
writing VRC01.protein

In [75]:
# To numpy arrays
complexMUT_sasas = np.asarray(complexMUT_sasas)
partnerA_MUT_sasas = np.asarray(partnerA_MUT_sasas)
partnerB_MUT_sasas = np.asarray(partnerB_MUT_sasas)

In [76]:
# interface sasa complex mut  (Phil, Phob, Total)
interface_asa_MUT = ((partnerA_MUT_sasas + partnerB_MUT_sasas) - complexMUT_sasas)/2

In [77]:
interface_asa_MUT_df = pd.DataFrame(interface_asa_MUT,columns=["if_phil","if_phob","if_total"],index= [pdb.name for pdb in pdbs_clean_mut])

In [78]:
interface_asa_MUT_df["sasa_phil"] = complexMUT_sasas.T[0]
interface_asa_MUT_df["sasa_phob"] = complexMUT_sasas.T[1]
interface_asa_MUT_df["sasa_total"] = complexMUT_sasas.T[2]

In [79]:
interface_asa_MUT_df.to_csv("../benchmark/VIH_sasa_mut.csv")

### sasa wildtype

In [80]:
outfolder = Path('popout_VIH')
if not outfolder.exists():
    outfolder.mkdir()
complexWT_sasas = list()
partnerA_WT_sasas = list()
partnerB_WT_sasas = list()
for i,pdb in enumerate(pdbs_clean_wt):
    
    pp = 'HL_G'
    
    # Calculate sasa for complex
    asa_data = run_pops(str(pdb.resolve()), outdir=outfolder, mode='total')
    pops_complex = POPS_parse2(asa_data, mode='total') 
    complexWT_sasas.append(list(pops_complex.values()))
    
    # Calculate sasa for partner A
    pdbA = get_pdb_chain(pdb.resolve(),partner=pp.split("_")[0],outdir=outfolder)
    asa_data = run_pops(pdbA, outdir=outfolder, mode='total')
    pops_A = POPS_parse2(asa_data, mode='total') 
    partnerA_WT_sasas.append(list(pops_A.values()))
    
    # Calculate sasa for partner B
    pdbB = get_pdb_chain(pdb.resolve(),partner=pp.split("_")[1],outdir=outfolder)
    asa_data = run_pops(pdbB, outdir=outfolder, mode='total')
    pops_B = POPS_parse2(asa_data, mode='total') 
    partnerB_WT_sasas.append(list(pops_B.values()))

writing VRC01.protein.G163A.mut.G163N.Repair4.clean_HL
writing VRC01.protein.G163A.mut.G163N.Repair4.clean_G
writing VRC01.protein.G163K.mut.G163N.Repair4.clean_HL
writing VRC01.protein.G163K.mut.G163N.Repair4.clean_G
writing VRC01.protein.G163T.mut.G163N.Repair4.clean_HL
writing VRC01.protein.G163T.mut.G163N.Repair4.clean_G
writing VRC01.protein.G165A.mut.G165S.Repair4.clean_HL
writing VRC01.protein.G165A.mut.G165S.Repair4.clean_G
writing VRC01.protein.G168A.mut.G168T.Repair4.clean_HL
writing VRC01.protein.G168A.mut.G168T.Repair4.clean_G
writing VRC01.protein.G176A.mut.G176F.Repair4.clean_HL
writing VRC01.protein.G176A.mut.G176F.Repair4.clean_G
writing VRC01.protein.G179A.mut.G179I.Repair4.clean_HL
writing VRC01.protein.G179A.mut.G179I.Repair4.clean_G
writing VRC01.protein.G218A.mut.G218R.Repair4.clean_HL
writing VRC01.protein.G218A.mut.G218R.Repair4.clean_G
writing VRC01.protein.G222A.mut.G222S.Repair4.clean_HL
writing VRC01.protein.G222A.mut.G222S.Repair4.clean_G
writing VRC01.prote

writing VRC01.protein.G88A.mut.G88K.Repair4.clean_HL
writing VRC01.protein.G88A.mut.G88K.Repair4.clean_G
writing VRC01.protein.G89A.mut.G89L.Repair4.clean_HL
writing VRC01.protein.G89A.mut.G89L.Repair4.clean_G


In [81]:
# To numpy arrays
complexWT_sasas = np.asarray(complexWT_sasas)
partnerA_WT_sasas = np.asarray(partnerA_WT_sasas)
partnerB_WT_sasas = np.asarray(partnerB_WT_sasas)

# interface sasa complex wt  (Phil, Phob, Total)
interface_asa_WT = ((partnerA_WT_sasas + partnerB_WT_sasas) - complexWT_sasas)/2

In [82]:
interface_asa_WT_df = pd.DataFrame(interface_asa_WT,columns=["if_phil","if_phob","if_total"],index= [pdb.name for pdb in pdbs_clean_wt])

In [83]:
interface_asa_WT_df["sasa_phil"] = complexWT_sasas.T[0]
interface_asa_WT_df["sasa_phob"] = complexWT_sasas.T[1]
interface_asa_WT_df["sasa_total"] = complexWT_sasas.T[2]

In [84]:
interface_asa_WT_df.to_csv("../benchmark/VIH_sasa_wt.csv")

# Data Final

In [17]:
DF_contact15_mut = pd.read_csv("../benchmark/VIH_contact15_mut.csv",index_col=0)
DF_contact15_wt = pd.read_csv("../benchmark/VIH_contact15_wt.csv",index_col=0)

DF_energetic_change_wt_mut = pd.read_csv("../benchmark/VIH_energetic_change_wt_mut.csv",index_col=0)
DF_energetic_change_mut_wt = pd.read_csv("../benchmark/VIH_energetic_change_mut_wt.csv",index_col=0)

DF_sasa_mut = pd.read_csv("../benchmark/VIH_sasa_mut.csv",index_col=0)
DF_sasa_wt = pd.read_csv("../benchmark/VIH_sasa_wt.csv",index_col=0)

In [105]:
import math
def Kd_to_deltaG(Kd):
    if Kd > 0:
        deltaG = (math.log10(Kd)*(1.98*298.15))/1000
        return deltaG
    else:
        deltaG = 0
        return deltaG
def deltaG_to_Kd(delg):
    Kd_value = math.exp((delg*1000)/(1.98*298.15))
    return Kd_value

In [18]:
# The mutant names are bad. Correct nomenclature is in REVERSE 
vih_mutant = vih_dataframe.loc[vih_dataframe['Mutation_type'] == 'ORIGINAL'].copy()
vih_mutant["chain/residue/mutant"] = ('G'+\
vih_dataframe.loc[vih_dataframe['Mutation_type'] == 'REVERSE']['Mutation'].str[1:]).values

# Sort values by new column addes
vih_mutant.sort_values(by="chain/residue/mutant",inplace=True)

In [19]:
DF_features_wt_mut = DF_contact15_mut.join([DF_energetic_change_wt_mut,DF_sasa_mut])
# add experimental ddG
DF_features_wt_mut['Exptal'] = vih_mutant["Exptal"].values

In [20]:
DF_features_wt_mut.drop(['number_of_residues','number_of_residues.1'],axis=1,inplace=True)

In [21]:
DF_features_wt_mut.to_csv('../benchmark/VIH_Test15.csv')